# **Fine Tunning DeepSeek-r1 - 8B Model**

In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-2xl8foa5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-2xl8foa5
  Resolved https://github.com/unslothai/unsloth.git to commit 1898b6d049d606ec88f3f9307172373776eec0f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.7.3-py3-none-any.whl size=298593 sha256=d1447d01f51b057f7ca9d9c266f069b8f5c29989f0750579dacce22d8d47a24d
  Stored in directory: /tmp/pip-ephem-wheel-cache-d7dovurj/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.7.3
    Uninstalling unsloth-2025.7.3:
      Successfully uninstalled unsloth-2025.7.3


In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
#Checking Huggingface Token
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [ ]:
# Step 1: Install wandb (if not already installed)
!pip install -q wandb

# Step 2: Import wandb
import wandb

# Step 3: Login securely using wandb API key
# Use input or environment variable (safer in shared notebooks)
import os

# Prompt user to paste token if not already set
if "WB_API" not in os.environ:
    os.environ["WB_API"] = input("Paste your WandB API Key here: ")

wandb.login(key=os.environ["WB_API"])

# Step 4: Initialize the wandb run
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset',
    job_type="training",
    anonymous="allow"
)


Paste your WandB API Key here: 760236e18a78d9225958849a73f811579238753f


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manojpendem19 (manojpendem19-madanapalle-institute-of-technology-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I'm trying to figure out what the cystometry would show for this 61-year-old woman. Let me break it down.

First, the patient has a history of involuntary urine loss when she coughs or sneezes, but she doesn't leak at night. That makes me think of stress urinary incontinence. Usually, that's due to the urethral sphincter not closing properly during activities that increase abdominal pressure, like coughing. But since she doesn't leak at night, it's less likely to be related to nocturnal enuresis, which is more common in children or people with severe sphincter weakness.

She underwent a gynecological exam and a Q-tip test. I'm not entirely sure what the Q-tip test involves, but I think it's a common diagnostic tool for stress incontinence. From what I remember, the Q-tip is a small, flexible catheter used to assess urethral function. The test probably involved inserting the Q-tip into her vagina to measure the closure pressure of the urethral sphincter. If the sphinct

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.7.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is sw

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["UNSLOTH_RETURN_LOGITS"]="1"
os.environ["TRITON_DISABLE_LINE_INFO"]="1"
os.environ["TRITON_INTERPRET"]="1"
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.932500
20,1.424200
30,1.404900
40,1.361700
50,1.391100
60,1.370700


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


In [ ]:
wandb.finish()


train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▂▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6686758448119808e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.2557
train/learning_rate,0.0
train/loss,1.3707


In [ ]:
import gradio as gr

# ✅ Reuse your already loaded model and tokenizer
# model and tokenizer must already be defined in previous cells
# You also already called: FastLanguageModel.for_inference(model)

# Prompt style from your original code
prompt_style = "### Instruction:\n{}\n\n### Response:"

# Inference function
def generate_response(question):
    try:
        inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1200,
            use_cache=True,
        )

        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_answer = response.split("### Response:")[-1].strip()
        return final_answer

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Gradio interface
demo = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(
        label="Enter Clinical Question",
        lines=5,
        placeholder="Type your medical prompt here...",
    ),
    outputs=gr.Textbox(label="Model Response"),
    title="🧠 Clinical QA with Your Unsloth Model",
    theme="default"
)

# Launch Gradio app
demo.launch(share=True)  # set share=False if not needed


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edb50d206b3e578c5a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
question = "A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue, and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative, gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium. What is the most likely predisposing factor for this patient's condition?"

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

<think>
Alright, let's break this down. We have a 59-year-old man with some classic symptoms of a bacterial infection: fever, chills, night sweats, and fatigue. These symptoms are pretty much textbook signs for an infection, especially with gram-positive organisms.

Now, looking at the blood culture results, it's really interesting. It says gram-positive, catalase-negative, gamma-hemolytic cocci in chains that don't grow in a 6.5% NaCl medium. Hmm, let's think about what kind of bacteria this could be. I'm trying to recall the different types of gram-positive cocci and their characteristics.

Staphylococcus aureus is a gram-positive, catalase-positive, gamma-hemolytic bacteria. But wait, the problem says catalase-negative. So, that doesn't match up. Okay, so scratch Staphylococcus aureus off the list.

What about Enterococcus faecalis? It's gram-positive, catalase-negative, and non-hemolytic, but it's not gamma-hemolytic like the problem suggests. Hmm, so that's not quite it either.

O

In [ ]:
new_model_online = "Manojkumar28/DeepSeek-R1-Medical-COT"
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)

('DeepSeek-R1-Medical-COT/tokenizer_config.json',
 'DeepSeek-R1-Medical-COT/special_tokens_map.json',
 'DeepSeek-R1-Medical-COT/chat_template.jinja',
 'DeepSeek-R1-Medical-COT/tokenizer.json')

In [ ]:
from huggingface_hub import create_repo, upload_folder

# 🔁 Reuse existing repo or create it
create_repo(repo_id="Manojkumar28/DeepSeek-R1-Medical-COT", private=True, exist_ok=True)

# 🚀 Upload folder (with progress and resume support)
upload_folder(
    folder_path="./DeepSeek-R1-Medical-COT",
    repo_id="Manojkumar28/DeepSeek-R1-Medical-COT",
    repo_type="model"
)


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Manojkumar28/DeepSeek-R1-Medical-COT/commit/4267f99e5abdb036401abd1193997ae49ea34f5e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4267f99e5abdb036401abd1193997ae49ea34f5e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Manojkumar28/DeepSeek-R1-Medical-COT', endpoint='https://huggingface.co', repo_type='model', repo_id='Manojkumar28/DeepSeek-R1-Medical-COT'), pr_revision=None, pr_num=None)

In [ ]:
!ls /content/DeepSeek-R1-Medical-COT


adapter_config.json	   README.md		    tokenizer.json
adapter_model.safetensors  special_tokens_map.json
chat_template.jinja	   tokenizer_config.json


In [ ]:
!zip -r DeepSeek-R1-Medical-COT.zip DeepSeek-R1-Medical-COT
from google.colab import files
files.download("DeepSeek-R1-Medical-COT.zip")


  adding: DeepSeek-R1-Medical-COT/ (stored 0%)
  adding: DeepSeek-R1-Medical-COT/special_tokens_map.json (deflated 69%)
  adding: DeepSeek-R1-Medical-COT/tokenizer.json (deflated 85%)
  adding: DeepSeek-R1-Medical-COT/adapter_model.safetensors (deflated 7%)
  adding: DeepSeek-R1-Medical-COT/README.md (deflated 45%)
  adding: DeepSeek-R1-Medical-COT/chat_template.jinja (deflated 75%)
  adding: DeepSeek-R1-Medical-COT/adapter_config.json (deflated 57%)
  adding: DeepSeek-R1-Medical-COT/tokenizer_config.json (deflated 96%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install bitsandbytes accelerate


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

# **Basics Of FineTunning**




**What is Transformer ?**

    A Transformer is a special type of AI model that reads the entire sentence
    all at once, instead of word by word. This allows it to understand the
    relationships between words in the sentence, helping it grasp the true
    meaning and context much better.

**Example:**

    In the sentence,

    “A dog is eating cheese because it was hungry,”

    the word  “it” refers to the dog.

    Transformers can easily understand this connection by
    looking at the whole sentence at once. Older models
    like RNNs and LSTMs read word by word and often
    struggled to correctly understand such relationships.
    

**How Transformers Work:**

    1. See the whole sentence at once: Transformers look
       at every word in the sentence together, not just one
       at a time.

    2. Use attention to focus: They use a method called
       “self-attention” to figure out which words are
       important and how they relate to each other.

    3. Understand word relationships: By paying attention
       to these connections, transformers understand the
       meaning and context better.





**What is an LLM (Large Language Model)?**

    An LLM is an AI model trained on a huge amount of text data. It learns
    patterns, grammar, facts, and reasoning from this data. When you give it
    text as input, the model processes it and generates a meaningful text
    output—like answering questions, writing essays, or having a conversation.


**How Large Language Models (LLMs) Work — Simply:**

    They read lots of text to learn how language works.

    They remember how words connect to each other.

    They look at your whole sentence at once to understand what you mean.

    Then, they write back a smart and relevant answer based on what they learned.

**Training Vs Fine Tuning**

    Training means building an AI model from scratch by teaching it with a
    large amount of data.

    Fine-tuning means taking an already trained model and modifying it using
    your own specific data to make it perform better for your particular needs.

**PEFT (Parameter-Efficient Fine-Tuning):**
    

    PEFT is the category or family of methods where:
      You don’t fine-tune the full model
      You only update a small portion

    Methods:
      1. LoRA 🔥
      2. Adapter Tuning
      3. Prefix Tuning
      4. Prompt Tuning
      5. BitFit


**UnSloth Library**

    Unsloth is a library used to improve training speed and reduce GPU
    memory pressure while fine-tuning large language models.
    
    Unsloth speeds up LLM fine-tuning and lowers GPU usage

**Steps**

    1. Installing the packages
    2. Initializing model and tokenizer
    3. Adding LoRA Adapters
    4. Data Preparation
    5. Training the Model
    6. Now to start the training of the model, using this trainer
    7. Interfacing the fine tunned Model

